In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
%matplotlib inline 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv('houses_lat_long_v4.csv')

In [3]:
df = df.dropna(subset=['LAT_LONG'])

In [4]:
df.columns

Index(['PID', 'ST_NUM', 'ST_NAME', 'ST_NAME_SUF', 'ZIPCODE', 'OWN_OCC',
       'AV_LAND', 'AV_BLDG', 'AV_TOTAL', 'GROSS_TAX', 'LAND_SF', 'YR_BUILT',
       'YR_REMOD', 'GROSS_AREA', 'LIVING_AREA', 'NUM_FLOORS', 'TOTAL_RMS',
       'BDRMS', 'FULL_BTH', 'HALF_BTH', 'BTH_STYLE', 'BTH_STYLE2',
       'BTH_STYLE3', 'KITCH_STYLE', 'HEAT_TYP', 'AC', 'FPLACE', 'INT_CND',
       'INT_FIN', 'VIEW', 'LAT_LONG'],
      dtype='object')

In [5]:
# First, look for null values so we can dropna
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

LAND_SF          19
YR_REMOD       7329
TOTAL_RMS        33
BTH_STYLE2    23961
BTH_STYLE3    58492
dtype: int64

In [6]:
# Get the coords
def split_coords(col):
    x = col.split(",")
    return x[0], x[1]

df['LAT'], df['LONG'] = zip(*df['LAT_LONG'].apply(split_coords))

In [7]:
df

,PID,ST_NUM,ST_NAME,ST_NAME_SUF,ZIPCODE,OWN_OCC,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,...,KITCH_STYLE,HEAT_TYP,AC,FPLACE,INT_CND,INT_FIN,VIEW,LAT_LONG,LAT,LONG
0,502555000,2,BEAVER,ST,2108,Y,1436300,1242500,2678800,2823455,...,M,W,N,0.0,G,N,A,"-71.07207,42.355618",-71.07207,42.355618
1,100001000,104 A 104,PUTNAM,ST,2128,Y,129900,411300,541200,570425,...,S,W,N,0.0,A,N,A,"-71.03256,42.379173",-71.03256,42.379173
2,100002000,197,LEXINGTON,ST,2128,N,121000,475100,596100,628289,...,M,F,C,0.0,A,N,A,"-71.03255,42.37957",-71.03255,42.37957
3,100003000,199,LEXINGTON,ST,2128,N,121400,421100,542500,571795,...,S,S,N,0.0,A,N,A,"-71.03248,42.379593",-71.03248,42.379593
4,100004000,201,LEXINGTON,ST,2128,N,121900,396400,518300,546288,...,S,W,N,0.0,A,N,A,"-71.03242,42.379616",-71.03242,42.379616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101662,1000319288,80-82,FENWOOD,RD,2115,Y,0,561500,561500,591821,...,M,F,C,0.0,G,N,A,"-71.10889,42.335903",-71.10889,42.335903
101663,1000319290,80-82,FENWOOD,RD,2115,Y,0,195300,195300,205847,...,M,F,C,0.0,G,N,A,"-71.10889,42.335903",-71.10889,42.335903
103067,1100253022,126,THORNTON,ST,2119,N,0,563900,563900,594351,...,M,F,C,0.0,G,N,A,"-71.09162,42.322926",-71.09162,42.322926
103068,1100253024,126,THORNTON,ST,2119,Y,0,614800,614800,647999,...,M,F,C,0.0,G,N,A,"-71.09162,42.322926",-71.09162,42.322926


In [10]:
df = df.fillna(0)

In [30]:
# features: bed, bath, sqft, lon/lat, garage, age (or year built), and lotsize

features = ['BDRMS', 'FULL_BTH', 'HALF_BTH', 'YR_BUILT', 'LIVING_AREA', 'LAT', 'LONG']
X = df[features] #Features
y = df['AV_TOTAL'] #Target
X

,BDRMS,FULL_BTH,HALF_BTH,YR_BUILT,LIVING_AREA,LAT,LONG
0,2.0,1.0,0.0,1900.0,1478.0,-71.07207,42.355618
1,6.0,3.0,0.0,1900.0,2202.0,-71.03256,42.379173
2,3.0,3.0,0.0,1920.0,2307.0,-71.03255,42.37957
3,5.0,3.0,0.0,1905.0,2268.0,-71.03248,42.379593
4,5.0,3.0,0.0,1900.0,2028.0,-71.03242,42.379616
...,...,...,...,...,...,...,...
101662,1.0,1.0,0.0,2016.0,620.0,-71.10889,42.335903
101663,1.0,1.0,0.0,2016.0,831.0,-71.10889,42.335903
103067,2.0,1.0,0.0,2017.0,1813.0,-71.09162,42.322926
103068,3.0,2.0,0.0,2017.0,1982.0,-71.09162,42.322926


In [49]:
# Apply weights
X['LAT']=X['LAT'].astype(float)
X['LONG']=X['LONG'].astype(float)
norm_x = (X - X.min())/(X.max() - X.min())

c:\users\sriram\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\sriram\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
norm_x.describe()

,BDRMS,FULL_BTH,HALF_BTH,YR_BUILT,LIVING_AREA,LAT,LONG
count,87158.000000,87158.000000,87158.000000,87158.000000,87158.000000,87158.000000,87158.000000
mean,0.203554,0.087708,0.052286,0.952573,0.090499,0.484975,0.558664
std,0.123810,0.042781,0.087566,0.023358,0.051372,0.218961,0.239329
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.111111,0.047619,0.000000,0.941526,0.050896,0.316276,0.338942
50%,0.166667,0.095238,0.000000,0.946482,0.079269,0.541698,0.608946
75%,0.277778,0.095238,0.166667,0.961348,0.122887,0.641381,0.753337
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.33, random_state=42)

In [51]:
# X_test.iloc[0]
X.dtypes
X.describe()
X.isnull().sum()

BDRMS          0
FULL_BTH       0
HALF_BTH       0
YR_BUILT       0
LIVING_AREA    0
LAT            0
LONG           0
dtype: int64

In [66]:
nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(X)

In [67]:
X.iloc[0]

BDRMS             2.000000
FULL_BTH          1.000000
HALF_BTH          0.000000
YR_BUILT       1900.000000
LIVING_AREA    1478.000000
LAT             -71.072070
LONG             42.355618
Name: 0, dtype: float64

In [69]:
distances, indices = nbrs.kneighbors([X.iloc[0].values])
print(indices)
print(distances)

[[    0 21147  2957  3055 31813]]
[[0.         0.05294841 1.0008234  1.4148704  1.41745588]]


In [72]:
df.iloc[indices[0]].AV_BLDG

0        1242500
22302     189300
3109      233300
3207      267800
33478     235700
Name: AV_BLDG, dtype: int64

In [22]:
X_test.shape

(28763, 8)

In [23]:
X_train.iloc[12112]

BDRMS                   5
FULL_BTH                5
HALF_BTH                2
LIVING_AREA          4289
YR_BUILT             1890
LAND_SF              2108
LAT            -71.066696
LONG            42.357723
Name: 6873, dtype: object

In [34]:
#######

X_sub = X[0:20:2]
X_sub


,BDRMS,FULL_BTH,HALF_BTH,LIVING_AREA,YR_BUILT,LAND_SF,LAT,LONG
0,2.0,1.0,0.0,1478.0,1900.0,1225.0,-71.07207,42.355618
2,3.0,3.0,0.0,2307.0,1920.0,1150.0,-71.03255,42.37957
4,5.0,3.0,0.0,2028.0,1900.0,1150.0,-71.03242,42.379616
6,9.0,3.0,0.0,4362.0,1900.0,2500.0,-71.03229,42.379665
8,11.0,3.0,0.0,4080.0,1900.0,2500.0,-71.032036,42.37976
10,5.0,3.0,0.0,3241.0,1900.0,2500.0,-71.031906,42.379807
12,6.0,3.0,0.0,3193.0,1900.0,2500.0,-71.03178,42.379856
14,8.0,3.0,0.0,3642.0,1900.0,2500.0,-71.03165,42.3799
16,5.0,2.0,0.0,3057.0,1899.0,5000.0,-71.03133,42.38002
18,8.0,3.0,0.0,4336.0,1900.0,2500.0,-71.031204,42.38007


In [36]:
nbrs = NearestNeighbors(n_neighbors=9, algorithm='ball_tree').fit(X_sub)
distances, indices = nbrs.kneighbors([X_sub.iloc[0].values])
print(indices)
print(distances)

[[0 2 1 6 5 7 4 9 3]]
[[   0.          555.1017944   832.62897027 2137.02363165 2175.73137179
  2511.6848931  2897.60487337 3129.50938683 3153.27353748]]


In [38]:
X_sub.iloc[7]

BDRMS                  8
FULL_BTH               3
HALF_BTH               0
LIVING_AREA         3642
YR_BUILT            1900
LAND_SF             2500
LAT            -71.03165
LONG             42.3799
Name: 14, dtype: object

In [32]:
distances, indices = nbrs.kneighbors([X_sub.iloc[7].values])
print(indices)
print(distances)

[[7 6 8 9 5 2 3 1 4]]
[[   0.           66.02272336  216.02546146 1359.00404709 1817.3073488
  2403.96505798 2436.25121857 2491.45379247 2639.38155635]]
